<a href="https://colab.research.google.com/github/Ruquin-11/EV_Vehicle_Charge/blob/main/ev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
pip install pandas numpy matplotlib seaborn scikit-learn

In [23]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [24]:
data=pd.read_csv('/content/ev dataset 1.csv')

In [25]:
data.head(10)

,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,September 30 2022,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,December 31 2022,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,January 31 2020,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,June 30 2022,Ferry,WA,Truck,0,0,0,"3,575","3,575",0.00
4,July 31 2021,Douglas,CO,Passenger,0,1,1,83,84,1.19
5,May 31 2018,Maui,HI,Passenger,1,0,1,59,60,1.67
6,November 30 2017,Northampton,PA,Passenger,0,1,1,66,67,1.49
7,March 31 2018,Nassau,NY,Passenger,1,0,1,37,38,2.63
8,March 31 2020,DeKalb,IN,Passenger,1,0,1,1,2,50.00
9,January 31 2019,Columbia,WA,Truck,0,0,0,"1,529","1,529",0.00


In [26]:
data.shape

(20819, 10)

In [27]:
data.info

<bound method DataFrame.info of                     Date          County State Vehicle Primary Use  \
0      September 30 2022       Riverside    CA           Passenger   
1       December 31 2022  Prince William    VA           Passenger   
2        January 31 2020          Dakota    MN           Passenger   
3           June 30 2022           Ferry    WA               Truck   
4           July 31 2021         Douglas    CO           Passenger   
...                  ...             ...   ...                 ...   
20814    January 31 2023      Rockingham    NH           Passenger   
20815       July 31 2020     Carson City    NV           Passenger   
20816   February 28 2022          Island    WA           Passenger   
20817   December 31 2020       San Diego    CA           Passenger   
20818   November 30 2019       Goochland    VA           Passenger   

      Battery Electric Vehicles (BEVs)  \
0                                    7   
1                                    1   
2                                    0   
3                                    0   
4                                    0   
...                                ...   
20814                                1   
20815                                1   
20816                              744   
20817                               14   
20818                                3   

      Plug-In Hybrid Electric Vehicles (PHEVs) Electric Vehicle (EV) Total  \
0                                            0                           7   
1                                            2                           3   
2                                            1                           1   
3                                            0                           0   
4                                            1                           1   
...                                        ...                         ...   
20814                                        0                           1   
20815                                        0                           1   
20816                                      350                       1,094   
20817                                        2                          16   
20818                                        1                           4   

      Non-Electric Vehicle Total Total Vehicles  Percent Electric Vehicles  
0                            460            467                       1.50  
1                            188            191                       1.57  
2                             32             33                       3.03  
3                          3,575          3,575                       0.00  
4                             83             84                       1.19  
...                          ...            ...                        ...  
20814                         14             15                       6.67  
20815                         10             11                       9.09  
20816                     62,257         63,351                       1.73  
20817                      2,724          2,740                       0.58  
20818                        271            275                       1.45  

[20819 rows x 10 columns]>

In [28]:
data.isnull().sum()

,0
Date,0
County,86
State,86
Vehicle Primary Use,0
Battery Electric Vehicles (BEVs),0
Plug-In Hybrid Electric Vehicles (PHEVs),0
Electric Vehicle (EV) Total,0
Non-Electric Vehicle Total,0
Total Vehicles,0
Percent Electric Vehicles,0


In [29]:
Q1 = data['Percent Electric Vehicles'].quantile(0.25)
Q3 = data['Percent Electric Vehicles'].quantile(0.75)
IQR = Q3 - Q1

lowerbound = Q1 - 1.5 * IQR
upperbound = Q3 + 1.5 * IQR
print('lowerbound:', lowerbound)
print('upperbound:', upperbound)

outliers = data[(data['Percent Electric Vehicles'] < lowerbound) | (data['Percent Electric Vehicles'] > upperbound)]
print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])

lowerbound: -3.5174999999999996
upperbound: 6.9025
Number of outliers in 'Percent Electric Vehicles': 2476


In [30]:
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data = data[data['Date'].notnull()]
data = data[data['Electric Vehicle (EV) Total'].notnull()]
data['County'] = data['County'].fillna('Unknown')
data['State'] = data['State'].fillna('Unknown')
print("Missing after fill:")
print(data[['County', 'State']].isnull().sum())

data.head()

Missing after fill:
County    0
State     0
dtype: int64


,Date,County,State,Vehicle Primary Use,Battery Electric Vehicles (BEVs),Plug-In Hybrid Electric Vehicles (PHEVs),Electric Vehicle (EV) Total,Non-Electric Vehicle Total,Total Vehicles,Percent Electric Vehicles
0,2022-09-30,Riverside,CA,Passenger,7,0,7,460,467,1.50
1,2022-12-31,Prince William,VA,Passenger,1,2,3,188,191,1.57
2,2020-01-31,Dakota,MN,Passenger,0,1,1,32,33,3.03
3,2022-06-30,Ferry,WA,Truck,0,0,0,"3,575","3,575",0.00
4,2021-07-31,Douglas,CO,Passenger,0,1,1,83,84,1.19


In [31]:
data['Percent Electric Vehicles'] = np.where(data['Percent Electric Vehicles'] > upperbound, upperbound,
                                 np.where(data['Percent Electric Vehicles'] < lowerbound, lowerbound, data['Percent Electric Vehicles']))

outliers = data[(data['Percent Electric Vehicles'] < lowerbound) | (data['Percent Electric Vehicles'] > upperbound)]
print("Number of outliers in 'Percent Electric Vehicles':", outliers.shape[0])

Number of outliers in 'Percent Electric Vehicles': 0
